In [ ]:
# ----------------- Import Libraries -----------------
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report

import shutil
import random

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Flatten, Input, BatchNormalization, Dense, Activation, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder

from copy import deepcopy

In [ ]:
# ----------------- ResNet50_Model Definition -----------------
def ResNet50_Model():
    input_shape = (224, 224, 3)
    resnet50 = tf.keras.applications.ResNet50(include_top=False, input_shape=input_shape, weights='imagenet')

    x = layers.GlobalAveragePooling2D()(resnet50.output)
    x = layers.Dense(units=1024, activation='relu')(x)
    x = layers.Dense(units=512, activation='relu')(x)
    x = layers.Dense(units=256, activation='relu')(x)
    x = layers.Dense(units=128, activation='relu')(x)
    x = layers.Dropout(0.5)(x)

    x = layers.Reshape((1, 1, 128))(x)
    x_max = layers.GlobalMaxPooling2D()(x)
    x_avg = layers.GlobalAveragePooling2D()(x)
    x = layers.Concatenate()([x_max, x_avg])
    x = layers.Dense(units=4, activation='softmax')(x)

    model = models.Model(inputs=resnet50.input, outputs=x, name='ResNet50_Model')
    return model

In [ ]:
model = ResNet50_Model()

# Compile models
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                loss='categorical_crossentropy', metrics=['accuracy'])

### Methods for Federated Proximal Algorithm

In [ ]:
def loss_classifier(predictions, labels):
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=predictions)
    return tf.reduce_mean(loss)


def loss_dataset(model, dataset, loss_f):
    loss = 0
    for idx, (features, labels) in enumerate(dataset):
        # Convert PyTorch tensors to NumPy and then to TensorFlow tensors
        features_np = features.detach().cpu().numpy()
        labels_np = labels.detach().cpu().numpy()

        features_tf = tf.convert_to_tensor(features_np, dtype=tf.float32)
        labels_tf = tf.convert_to_tensor(labels_np, dtype=tf.int32)

        predictions = model(features_tf)
        loss += loss_f(predictions, labels_tf)
    loss /= (idx + 1)
    return loss


def accuracy_dataset(model, dataset):
    correct = 0
    total = 0
    for features, labels in dataset:
        # Convert PyTorch tensors to NumPy and then to TensorFlow tensors
        features_np = features.detach().cpu().numpy()
        labels_np = labels.detach().cpu().numpy()

        features_tf = tf.convert_to_tensor(features_np, dtype=tf.float32)
        labels_tf = tf.convert_to_tensor(labels_np, dtype=tf.int32)

        predictions = model(features_tf)
        predicted = tf.argmax(predictions, axis=1, output_type=tf.int32)
        correct += tf.reduce_sum(tf.cast(tf.equal(predicted, labels_tf), tf.int32)).numpy()
        total += labels_tf.shape[0]

    accuracy = 100 * correct / total
    return accuracy


def train_step(model, model_0, mu, optimizer, train_data, loss_f):
    total_loss = 0
    for idx, (features, labels) in enumerate(train_data):
        # Convert PyTorch tensors to NumPy and then to TensorFlow tensors
        features_np = features.detach().cpu().numpy()
        labels_np = labels.detach().cpu().numpy()

        features_tf = tf.convert_to_tensor(features_np, dtype=tf.float32)
        labels_tf = tf.convert_to_tensor(labels_np, dtype=tf.int32)

        with tf.GradientTape() as tape:
            predictions = model(features_tf)
            loss = loss_f(predictions, labels_tf)
            loss += mu / 2 * difference_models_norm_2(model, model_0)

        total_loss += loss
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    return total_loss / (idx + 1)


def local_learning(model, mu, optimizer, train_data, epochs, loss_f):
    # Clone the model instead of using deepcopy
    model_0 = tf.keras.models.clone_model(model)
    model_0.set_weights(model.get_weights())  # Set weights to be identical initially

    for e in range(epochs):
        local_loss = train_step(model, model_0, mu, optimizer, train_data, loss_f)

    return local_loss


def difference_models_norm_2(model_1, model_2):
    norm = tf.reduce_sum([tf.reduce_sum(tf.square(w1 - w2)) for w1, w2 in zip(model_1.trainable_variables, model_2.trainable_variables)])
    return norm

def set_to_zero_model_weights(model):
    for layer_weights in model.trainable_variables:
        layer_weights.assign(tf.zeros_like(layer_weights))

def average_models(model, clients_models_hist, weights):
    set_to_zero_model_weights(model)
    for k, client_hist in enumerate(clients_models_hist):
        for idx, layer_weights in enumerate(model.trainable_variables):
            contribution = client_hist[idx] * weights[k]
            layer_weights.assign_add(contribution)


### Federated Proximal Algorithm

In [5]:
def FedProx(model, training_sets, n_iter, testing_sets, mu=0, epochs=5, lr=0.01, decay=1):
    # Verify that `model` is a Keras model instance
    if not isinstance(model, tf.keras.Model):
        raise TypeError("The provided model is not a TensorFlow Keras model. Please provide a valid Keras model.")

    loss_f = loss_classifier
    K = len(training_sets)
    n_samples = sum([len(db) for db in training_sets])
    weights = [len(db) / n_samples for db in training_sets]
    print("Clients' weights:", weights)

    # Initialize history lists for training and testing
    train_loss_hist = []
    train_acc_hist = []
    test_loss_hist = []
    test_acc_hist = []
    models_hist = []

    for i in range(n_iter):
        clients_params = []
        clients_losses = []
        clients_accuracies = []

        for k in range(K):
            # Clone the model and set weights for local training
            local_model = tf.keras.models.clone_model(model)
            local_model.set_weights(model.get_weights())

            local_optimizer = tf.keras.optimizers.SGD(learning_rate=lr)

            # Perform local training and track the loss
            local_loss = local_learning(local_model, mu, local_optimizer, training_sets[k], epochs, loss_f)
            clients_losses.append(local_loss)

            # Track training accuracy for the client
            train_acc = accuracy_dataset(local_model, training_sets[k])
            clients_accuracies.append(train_acc)

            # Store model parameters (deep copy to ensure immutability)
            clients_params.append([tf.identity(tens_param) for tens_param in local_model.trainable_variables])

        # Average the local models into the global model
        average_models(model, clients_params, weights=weights)
        models_hist.append(deepcopy(clients_params))

        # Collect metrics for this iteration
        train_loss_hist.append(clients_losses)
        train_acc_hist.append(clients_accuracies)

        # Compute testing metrics using the global model
        test_loss_hist.append([loss_dataset(model, dl, loss_f).numpy() for dl in testing_sets])
        test_acc_hist.append([accuracy_dataset(model, dl) for dl in testing_sets])

        # Update learning rate by decay factor
        lr *= decay
        print(f'====> i: {i+1} Server Test Accuracy: {test_acc_hist[-1]}')

    return model, train_loss_hist, train_acc_hist, test_loss_hist, test_acc_hist



### To plot Training and Testing, accuracy and loss of trained model
### Save the result in a text file

In [6]:
import matplotlib.pyplot as plt

def plot_acc_loss(title: str, train_loss_hist: list, train_acc_hist: list,
                  test_loss_hist: list, test_acc_hist: list):
    plt.figure(figsize=(15, 5))  # Make the plot wider

    # Plot training loss
    plt.subplot(2, 2, 1)
    lines = plt.plot(train_loss_hist)
    plt.title("Training Loss")
    plt.legend(lines, [f"C{i+1}" for i in range(len(train_loss_hist[0]))])

    # Plot training accuracy
    plt.subplot(2, 2, 2)
    lines = plt.plot(train_acc_hist)
    plt.title("Training Accuracy")
    plt.legend(lines, [f"C{i+1}" for i in range(len(train_acc_hist[0]))])

    # Plot testing loss
    plt.subplot(2, 2, 3)
    lines = plt.plot(test_loss_hist)
    plt.title("Testing Loss")
    plt.legend(lines, [f"C{i+1}" for i in range(len(test_loss_hist[0]))])

    # Plot testing accuracy
    plt.subplot(2, 2, 4)
    lines = plt.plot(test_acc_hist)
    plt.title("Testing Accuracy")
    plt.legend(lines, [f"C{i+1}" for i in range(len(test_acc_hist[0]))])

    plt.suptitle(title)
    plt.tight_layout()
    plt.show()


def save_history_to_file(filename, n_iter, train_loss_hist, train_acc_hist,
                         test_loss_hist, test_acc_hist):
    with open(filename, 'w') as f:
        f.write("FedProx Training Results\n")
        f.write("="*50 + "\n")
        f.write(f"Number of iterations: {n_iter}\n\n")

        # Write the history for each iteration
        for i in range(n_iter):
            f.write(f"Iteration {i+1}:\n")
            f.write(f"Train Loss: {train_loss_hist[i]}\n")
            f.write(f"Train Accuracy: {train_acc_hist[i]}\n")
            f.write(f"Test Loss: {test_loss_hist[i]}\n")
            f.write(f"Test Accuracy: {test_acc_hist[i]}\n")
            f.write("-"*50 + "\n")

    print(f"Training history saved to {filename}")

### Load the NON-IID dataset

In [ ]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Directories for each client
split_dirs = [
    "NON_IID/Model_1",
    "NON_IID/Model_2",
    "NON_IID/Model_3",
    "NON_IID/Model_4"
]

# Number of clients
n_clients = len(split_dirs)

# Updated image transformations: Normalize first, then permute
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),          # Convert image to tensor format (C, H, W)
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize
    transforms.Lambda(lambda x: x.permute(1, 2, 0))  # Permute to (H, W, C)
])

def get_tumor_dataloaders(split_dirs, batch_size=25, shuffle=True):
    """
    Returns data loaders for all clients for both training and testing sets.
    """
    tumor_iid_train_dls = []
    tumor_iid_test_dls = []

    for client_idx, client_dir in enumerate(split_dirs):
        # Get the directory for the current client
        train_dir = os.path.join(client_dir, 'train')
        test_dir = os.path.join(client_dir, 'test')

        # Check if the directories exist
        if not os.path.exists(train_dir) or not os.path.exists(test_dir):
            print(f"Directory not found for client {client_idx + 1}:")
            print(f"Train dir: {train_dir}")
            print(f"Test dir: {test_dir}")
            continue

        # Load training data for the current client
        train_dataset = datasets.ImageFolder(train_dir, transform=transform)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)

        # Load testing data for the current client
        test_dataset = datasets.ImageFolder(test_dir, transform=transform)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

        # Append the dataloaders for the current client to the list
        tumor_iid_train_dls.append(train_loader)
        tumor_iid_test_dls.append(test_loader)

    return tumor_iid_train_dls, tumor_iid_test_dls

# Get the training and testing data loaders
Tumor_iid_train_dls, Tumor_iid_test_dls = get_tumor_dataloaders(split_dirs, batch_size=25)

# Checking the sizes of the images in the data loaders to verify the shape
for batch_idx, (images, labels) in enumerate(Tumor_iid_train_dls[0]):  # Checking for client 1
    print(f"Batch {batch_idx} image sizes: {images.size()}")  # Should print torch.Size([25, 224, 224, 3])
    break  # Check only the first batch


### Execute the Fed Average using ResNet50+CBAM model

In [ ]:
n_iter = 10
# Execute FedProx with the new 4-client setup
model, train_loss, train_acc, test_loss, test_acc = FedProx(
    model, Tumor_iid_train_dls, n_iter, Tumor_iid_test_dls, mu=0.0, epochs=1, lr=0.0001
)

In [ ]:
# File to save the results
output_file = "FedAvg_ResNet50.txt"
# Save the history to a file
save_history_to_file(output_file, n_iter, train_loss, train_acc, test_loss, test_acc)


### Evaluate model with IID testing data (From original Kaggle testing data - 1311 images)
### IID testing data is used, To check the model performance

In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import tensorflow as tf
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, classification_report
import numpy as np
import os

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),          # Convert image to tensor format (C, H, W)
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize
    transforms.Lambda(lambda x: x.permute(1, 2, 0))  # Permute to (H, W, C)
])

# Load the testing dataset
test_dir = 'BrainTumor_MRI/Testing'
test_dataset = datasets.ImageFolder(test_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Class names mapping (adjust if necessary)
class_names = test_dataset.classes


In [ ]:
# Function to evaluate the global model on the testing dataset
def evaluate_global_model(model, test_loader):
    all_preds = []
    all_labels = []
    total_loss = 0

    for features, labels in test_loader:
        # Convert to TensorFlow tensors
        features_np = features.detach().cpu().numpy()
        labels_np = labels.detach().cpu().numpy()

        features_tf = tf.convert_to_tensor(features_np, dtype=tf.float32)
        labels_tf = tf.convert_to_tensor(labels_np, dtype=tf.int32)

        # Get predictions
        predictions = model(features_tf)
        predicted = tf.argmax(predictions, axis=1, output_type=tf.int32)

        # Accumulate predictions and labels
        all_preds.extend(predicted.numpy())
        all_labels.extend(labels_np)

        # Compute loss
        loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels_tf, logits=predictions)
        total_loss += tf.reduce_mean(loss).numpy()

    # Compute metrics
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    conf_matrix = confusion_matrix(all_labels, all_preds)

    print(f"\nConfusion Matrix:\n{conf_matrix}")
    print(f"\nAccuracy: {accuracy * 100:.2f}%")
    print(f"Loss: {total_loss / len(test_loader):.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")

    # Print detailed classification report
    print("\nClassification Report:\n", classification_report(all_labels, all_preds, target_names=class_names))



In [ ]:
# Evaluate the model on the testing dataset
evaluate_global_model(model, test_loader)